<a href="https://colab.research.google.com/github/shi-star/Text_Sentiment_Analysis_movie_review/blob/main/Sentiment__TextAnalysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [64]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [65]:
path = '/content/drive/MyDrive/archive.zip'

In [66]:
import pandas as pd
import nltk
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer
import numpy as np
import re
from sklearn.feature_extraction.text import TfidfVectorizer

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


1. We have imported NLTK (Natural Language ToolKit), for stopping, stemming, lemmatization
2. nltk.download('punkt') # default download,It is downloaded default as available in the package
3. nltk.download('stopwords') # downloading the stopwords to be used in the stopping process
4. nltk.download('wordnet') # downloading wordnet(is like imagenet for text) text data for stemming and lemmatization purpose
5. from nltk.corpus import stopwords # imports the stopwords we can use
6. from nltk.stem.wordnet import WordNetLemmatizer # Lemmatizer 
7. import re # Regular Expression, RegEx
8. from sklearn.feature_extraction.text import TfidfVectorizer (to calculate TF, IDF)

In [67]:
txt = pd.read_csv(path, nrows = 1000)

Although the data has 25000 rows, but it didn't run well so I took only 1000 rows of the data 

In [68]:
txt.head()

,sentiment,review
0,Positive,With all this stuff going down at the moment w...
1,Positive,'The Classic War of the Worlds' by Timothy Hin...
2,Negative,The film starts with a manager (Nicholas Bell)...
3,Negative,It must be assumed that those who praised this...
4,Positive,Superbly trashy and wondrously unpretentious 8...


Glimpse of the Data.

In [69]:
import numpy as np



Forgot to import :)

In [70]:
txt.shape

(1000, 2)

We have 1000 rows and 2 columns, one is target column.

In [71]:
txt['sentiment'].value_counts()

Negative    518
Positive    482
Name: sentiment, dtype: int64

We can see we have balanced data . almost an equal number of both the sentiments.

In [72]:
dict1 = {'Positive':1, 'Negative':0}

So let's change them into numerical so that it will be easy to process further 

In [73]:
txt['sentiment'] = txt['sentiment'].map(dict1)

In [74]:
txt['sentiment'].value_counts()

0    518
1    482
Name: sentiment, dtype: int64

In [75]:
txt['review'][1]

"'The Classic War of the Worlds' by Timothy Hines is a very entertaining film that obviously goes to great effort and lengths to faithfully recreate H. G. Wells' classic book. Mr. Hines succeeds in doing so. I, and those who watched his film with me, appreciated the fact that it was not the standard, predictable Hollywood fare that comes out every year, e.g. the Spielberg version with Tom Cruise that had only the slightest resemblance to the book. Obviously, everyone looks for different things in a movie. Those who envision themselves as amateur 'critics' look only to criticize everything they can. Others rate a movie on more important bases,like being entertained, which is why most people never agree with the 'critics'. We enjoyed the effort Mr. Hines put into being faithful to H.G. Wells' classic novel, and we found it to be very entertaining. This made it easy to overlook what the 'critics' perceive to be its shortcomings.'"

We can see second row of the data, and we can see each row has a lot of text in it. 

#**1.Creating Columns for TF-IDF table**

In [ ]:
text = ''

for i in txt['review']:
  text += re.sub('[^a-z ]', '', i.strip().lower())+' '
print(text)

Here we have taken each row and then applied the regex function on each row to remove all the character except (a-z and a space).

# **Tokenisation**

In [77]:
nltk.word_tokenize(text)

['with',
 'all',
 'this',
 'stuff',
 'going',
 'down',
 'at',
 'the',
 'moment',
 'with',
 'mj',
 'ive',
 'started',
 'listening',
 'to',
 'his',
 'music',
 'watching',
 'the',
 'odd',
 'documentary',
 'here',
 'and',
 'there',
 'watched',
 'the',
 'wiz',
 'and',
 'watched',
 'moonwalker',
 'again',
 'maybe',
 'i',
 'just',
 'want',
 'to',
 'get',
 'a',
 'certain',
 'insight',
 'into',
 'this',
 'guy',
 'who',
 'i',
 'thought',
 'was',
 'really',
 'cool',
 'in',
 'the',
 'eighties',
 'just',
 'to',
 'maybe',
 'make',
 'up',
 'my',
 'mind',
 'whether',
 'he',
 'is',
 'guilty',
 'or',
 'innocent',
 'moonwalker',
 'is',
 'part',
 'biography',
 'part',
 'feature',
 'film',
 'which',
 'i',
 'remember',
 'going',
 'to',
 'see',
 'at',
 'the',
 'cinema',
 'when',
 'it',
 'was',
 'originally',
 'released',
 'some',
 'of',
 'it',
 'has',
 'subtle',
 'messages',
 'about',
 'mjs',
 'feeling',
 'towards',
 'the',
 'press',
 'and',
 'also',
 'the',
 'obvious',
 'message',
 'of',
 'drugs',
 'are',
 

**We have tokenised the data so that each word is now in an element of a list. it returns individual words from the given text in the form of a list, this is tokenizer
We are using word_tokenize function of nltk here, but alternatively we can also use .split()**

In [78]:
word = list(set(nltk.word_tokenize(text)))
print(len(word))

20408


**We have taken the set here so that all the duplicates will be removed. And we can see we have 20408 words i.e columns**

# **Removing Stopwords**

In [79]:
filtered = list(set([w for w in word if not w in stopwords.words('english')]))
print(len(filtered))
print(filtered)

20275
['scan', 'condition', 'blight', 'horrid', 'reams', 'presidents', 'newport', 'barker', 'stealing', 'wvs', 'offshoot', 'annual', 'kitagawa', 'masterpiece', 'nakadei', 'rohmer', 'freeze', 'custers', 'oeuvre', 'hentai', 'rebellious', 'castros', 'theories', 'storys', 'turners', 'wendy', 'encourages', 'hotsoul', 'sherry', 'englishman', 'adulation', 'secretary', 'rooted', 'bacharach', 'approved', 'fend', 'std', 'liberties', 'centers', 'snare', 'fellow', 'mean', 'empire', 'subtle', 'splash', 'duran', 'ottaviano', 'jagged', 'net', 'slime', 'sweeney', 'forgettable', 'cheaply', 'ludvig', 'beggars', 'predictable', 'penis', 'hull', 'flynns', 'passion', 'parting', 'switched', 'step', 'nihilism', 'managed', 'inserted', 'lucien', 'outer', 'ornamental', 'sweeping', 'hamliton', 'skilled', 'hoven', 'makingof', 'jumping', 'drawnout', 'succumbed', 'slut', 'arabic', 'introverted', 'across', 'hazares', 'throughhe', 'designated', 'fourth', 'occasionalb', 'rare', 'moviewas', 'manner', 'western', 'pecking

**Here we have filtered the data so that all the irrelevant words will be removed. And now we can see we have 20275 columns left**

# **Lammetization**

In [80]:
wordnet_lammetizer = WordNetLemmatizer()

In [81]:
wordnet_lammetizer_v = []
for i in filtered:
  wordnet_lammetizer_v.append(wordnet_lammetizer.lemmatize(i,'v'))

wordnet_v = list(set(wordnet_lammetizer_v))
print(len(wordnet_v))

17201


**After lammetization we are left with 17208 columns or unique words.As we have taken set of the lammetized word, lemmatize to remove repetitive words like play, played to reduce this number.**

Till here we have created columns of unique words , using Lammetisation, tokenisation. Now we will create rows but will not take set as we do not want to remove duplicates from the rows.

# **2.Creating Rows to calculate TF-IDF**

In [82]:
train_file_wordnet_v = []


for line in txt['review']:
    
    x = re.sub('[^a-z ]', '', line.strip().lower())
    tokens = nltk.word_tokenize(x)
    filtered_tokens = [w for w in tokens if not w in stopwords.words('english')]
    stemmed_tokens = ''
    for item in filtered_tokens:
        stemmed_tokens += wordnet_lammetizer.lemmatize(item, 'v')+' '
    train_file_wordnet_v.append(stemmed_tokens[:-1]) # v means verb, we will consider play --> play and played --> play as same 
# wordnet_v is the new list which has unique non-stopwords and lemmatized words


print(len(train_file_wordnet_v))
print(train_file_wordnet_v[1])

1000
classic war worlds timothy hines entertain film obviously go great effort lengths faithfully recreate h g well classic book mr hines succeed watch film appreciate fact standard predictable hollywood fare come every year eg spielberg version tom cruise slightest resemblance book obviously everyone look different things movie envision amateur critics look criticize everything others rate movie important baseslike entertain people never agree critics enjoy effort mr hines put faithful hg well classic novel find entertain make easy overlook critics perceive shortcomings


**We have cleaned the rows, like removed the stopwords, applied the tokenisation and lammetization.**

In [88]:
x = train_file_wordnet_v

**Now let's do the train_test split**

In [90]:
len(x)

1000

In [92]:
y = txt['sentiment']

In [91]:
from sklearn.model_selection import train_test_split

In [93]:
x_train, x_test, y_train, y_test = train_test_split(x,y, test_size=0.3, random_state=0)

**Here we have created the object of the TF_iDF to calculate TF-IDF**

In [94]:
vectorizer = TfidfVectorizer()

In [95]:
X_train_tfidf = vectorizer.fit_transform(x_train)

**We have fit_transform the x_train**

In [96]:
print(X_train_tfidf.shape)

(700, 13766)


In [97]:
print(type(X_train_tfidf))

<class 'scipy.sparse.csr.csr_matrix'>


In [98]:
X_train_tfidf1 = X_train_tfidf.toarray()

**As model can not take anyother format than pd dataframe or array. so changed it to the array**

In [99]:
X_test_tfidf = vectorizer.transform(x_test)

**Transform the X_test**

In [100]:
X_test_tfidf1 = X_test_tfidf.toarray()

**Imported the model , we are using Random Forest , as it is a binary classification**

In [101]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import GridSearchCV

In [102]:
model_parameters = {
        'n_estimators':[ 100],
        'criterion':['gini'],
        'max_depth': [3, 5]
    }

In [103]:
model = RandomForestClassifier(random_state=1)
gscv = GridSearchCV(estimator=model, 
                    param_grid=model_parameters, 
                    cv=5, 
                    verbose=1, 
                    n_jobs=-1,
                    scoring='roc_auc')

gscv.fit(X_train_tfidf1, y_train)

Fitting 5 folds for each of 2 candidates, totalling 10 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed:    5.9s finished


GridSearchCV(cv=5, error_score=nan,
             estimator=RandomForestClassifier(bootstrap=True, ccp_alpha=0.0,
                                              class_weight=None,
                                              criterion='gini', max_depth=None,
                                              max_features='auto',
                                              max_leaf_nodes=None,
                                              max_samples=None,
                                              min_impurity_decrease=0.0,
                                              min_impurity_split=None,
                                              min_samples_leaf=1,
                                              min_samples_split=2,
                                              min_weight_fraction_leaf=0.0,
                                              n_estimators=100, n_jobs=None,
                                              oob_score=False, random_state=1,
                                   

In [104]:
print('The best parameter are -', gscv.best_params_)

The best parameter are - {'criterion': 'gini', 'max_depth': 5, 'n_estimators': 100}


In [105]:
print(gscv.best_score_)
print(gscv.best_estimator_)
print(gscv.scorer_)

0.8394719043870547
RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=5, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=1, verbose=0,
                       warm_start=False)
make_scorer(roc_auc_score, needs_threshold=True)


In [107]:
print('AUC on test by gscv =', roc_auc_score(y_true=y_test,
                                                        y_score=gscv.predict_proba(X_test_tfidf1)[:, 1]))

AUC on test by gscv = 0.8796390950708921


We can see the Roc_auc score is 87% using Random Forest with the help of the Grid Search CV